In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import os
from PIL import Image
from io import BytesIO
import time
import random

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print(tf.__version__)

def mamon_videoFightModel2(tf, wight='/content/drive/MyDrive/archive (2)/mamonbest947oscombo-drive.hdfs'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2

    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False, weights='imagenet', input_shape=(160, 160, 3))

    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())

    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn, input_shape=(30, 160, 160, 3)))
    model.add(layers.LSTM(30, return_sequences=True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = optimizers.RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model

# Assuming mamon_videoFightModel2 is your model definition
# If you have not defined it in the provided code, replace it with your actual model definition
model22 = mamon_videoFightModel2(tf)

# Define your learning rate
your_learning_rate = 0.001  # Replace with your desired learning rate

# Update the optimizer creation by replacing lr with learning_rate
optimizer = tf.keras.optimizers.Adam(learning_rate=your_learning_rate)

# Compile the model with the updated optimizer
model22.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

def video_mamonreader(cv2, filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i = 0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval, frame = vc.read()
    else:
        rval = False
    frm = resize(frame, (160, 160, 3))
    frm = np.expand_dims(frm, axis=0)
    if np.max(frm) > 1:
        frm = frm / 255.0
    frames[i][:] = frm
    i += 1
    print("reading video")
    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame, (160, 160, 3))
        frm = np.expand_dims(frm, axis=0)
        if np.max(frm) > 1:
            frm = frm / 255.0
        frames[i][:] = frm
        i += 1
    return frames

def pred_fight(model, video, accuracy=0.9):
    pred_test = model.predict(video)
    if pred_test[0][1] >= accuracy:
        return True, pred_test[0][1]
    else:
        return False, pred_test[0][1]

def main_fight(folder_path):
    # Get a list of all video files in the specified folder
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the specified folder.")
        return

    # Choose a random video file
    random_video = random.choice(video_files)
    video_path = os.path.join(folder_path, random_video)

    # Process the randomly selected video
    vid = video_mamonreader(cv2, video_path)
    datav = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    datav[0][:][:] = vid
    millis = int(round(time.time() * 1000))
    print(millis)
    fight, precent = pred_fight(model22, datav, accuracy=0.65)
    millis2 = int(round(time.time() * 1000))
    print(millis2)
    res_mamon = {'fight': fight, 'precentegeoffight': str(precent)}
    res_mamon['processing_time'] = str(millis2 - millis)
    return res_mamon

# Specify the folder containing your video files
folder_path = '/content/drive/MyDrive/Real Life Violence Dataset/Violence'

# Call the main_fight function with the specified folder path
res = main_fight(folder_path)

print(res)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 798974518223299219
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7853252830071561394
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
2.15.0


<ipython-input-64-0fe5eda27080>:73: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frames = np.zeros((30, 160, 160, 3), dtype=np.float)


(30, 160, 160, 3)
reading video


<ipython-input-64-0fe5eda27080>:119: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datav = np.zeros((1, 30, 160, 160, 3), dtype=np.float)


1705561179316
1/1 [==============================] - 1s 637ms/step
1705561180030
{'fight': True, 'precentegeoffight': '0.9659322', 'processing_time': '714'}
